# TMS Data Management

This script reads in raw data from participant sessions (i.e. output from the `PsychoPy` test instrument software). It reformats it into a single `.csv` file containing item-level information for all participants and sessions. Here, "item-level" means that each observation (row in the dataset) represents one person's observation of (response to) one stimulus prompt (question), necessarily under one TMS condition (brain region).

Before running this code, follow the **Setup instructions** below to ensure that raw data is present and discoverable. Also, pay attention to the **Options**: some aspects of this script, such as whether to randomly anonymize treatment conditions and/or content domains (types of questions), are configurable. The options also specify the names of the input and output files.

The output directory is called `processed-data-[x]` in the working directory, where `x` is the (configurable) version number. If anonymization is used, the masking scheme is given in `processed-data-[x]/secret-keys`. The actual dataframe is the `tms-functional-data-v[x].csv` file in the output directory.

## Setup instructions: getting data into the filesystem
- From Google Drive, download the [_functional\_data_](https://drive.google.com/drive/folders/1NCyoM-Uj-h0-F7_a6ywdBssdFzRjxvYG) folder. This will automatically zip the contents.
- Put the above two files into your working directory and `unzip` the first one.
- Rename the new directory from `functional_data` to `test-data-[x]`, e.g. I'm using `test-data-1`.
- In the cell below (under __Options__), redefine the `input_version` variable to whatever you chose `x` to be (so mine is set to 1). Also pick an output version number and set the variable `output_version` accordingly (I chose 1).
- Participant 00001 has two copies of one of the `.csv` files. One is corrupted, so we rename it accordingly:
```
$ cd test-data-[x]/00001
$ mv 00001_codetms_version2_2023-04-04_11h35.48.881block_1.csv 00001_codetms_version2_2023-04-04_11h35.48.881block_1-corrupted.csv
```
- Download the [Logistics](https://docs.google.com/spreadsheets/d/1_iH6ziOTDaZRM1K2_0EuHF3drL8ncgBdipbqAYnCkEs/edit#gid=0) sheet into your new `test-data-[x]` directory. Rename it to `logistics.csv`
- Download the [post survey results](https://docs.google.com/spreadsheets/d/1QSpX9H8o5w8oLm6xEd_R7H0eWunDcEqh2Mt9uwmGUhw/edit?usp=sharing) sheet into your new `test-data-[x]` directory. Rename it to `post_survey_results.csv`
- Now you should be good to go!

## Options (you can change these)

In [1]:
verbose = False # whether to print verbose debugging output (set to False if using anonymization)

input_version = 1 # input test data version
output_version = 3 # output version (for distinguish different anonymizations, etc.)

output = False # whether to output the results to a file

reset_indices = True # whether to change the ranges of stimulus indices to be contiguous (original indices are still retained)

mask_flags = {"treatment_condition" : True, # whether to anonymize the treatment condition (vertex, sma, m1)
              "stimulus_domain" : True, # whether to anonymize the stimulus domain (list/array, tree, shepard-metzler, PSVT:R II, code)
             }

## Miscellaneous code setup

In [2]:
import pandas as pd
import numpy as np
import os
import random

In [3]:
datadirname = "test-data-%d" % input_version # the name of the directory where all the data is stored

# values needed to calculate random mask for anonymization
mask_vals = {"treatment_condition" : ["sma", "m1", "vertex"],
             "stimulus_domain" : ["list/array", "tree", "shepard-metzler", "PSVT:R II", "code"]}

# starting points of the mask output ranges
mask_start = {"treatment_condition" : 'X',
              "stimulus_domain" : 'A'}

## Gather raw data from filesystem

In [4]:
# dictionary mapping participant id to set of corresponding filenames
data_files = {}

# find all the csv files, grouped by participant
for root, dirs, files in os.walk(datadirname):
    for fname in files:
        # for each participant and session, the data is stored in several
        # different file formats; we just pick one and stay consistent
        if fname[-11:] == "block_1.csv":
            id = str(fname[:5])
            fullname = os.path.join(root, fname)
            if id in data_files.keys():
                data_files[id].add(fullname)
            else:
                data_files[id] = {fullname}

# remove any participants without enough data; two sessions is ok but
# we can't do anything if there's just one (not applicable for final data)
data_files = {id : fileset for id, fileset in data_files.items() if len(fileset) >= 2}

# print things out just to check
if verbose:
    for (id, fileset) in data_files.items():
        print(id, ":")
        for file in fileset:
            print("\t", file)

In [5]:
# make dictionary mapping participant to list of session types in chronological order

sessionsdf = pd.read_csv(datadirname + "/logistics.csv")

# all metadata for the participant
sessions = {id : sessionsdf.loc[sessionsdf["Participant_ID"] == int(id)]
            for id in data_files.keys()}

# only keep session type data
sessions = {id : 
            [sessions[id].reset_index()._get_value(0, ("Session Type %d" % k))
             for k in range(1, len(fnames) + 1)]
            for id, fnames in data_files.items()}

# split experimental condition and test version
sessions = {id : [info.split(", version ") for info in sessions[id]]
            for id in sessions.keys()}

# store as a dict of dicts rather than lists
sessions = {id : [{"condition" : ls[0], "version" : int(ls[1])} for ls in ls]
            for id, ls in list(sessions.items())}

if verbose:
    display(sessions)

In [6]:
# given the session info and fileset for a participant, combine and store everything we need
def categorize(id, infolist, fileset):
    dict = {}
    
    # for each session
    for i, info in enumerate(infolist):
        v = info["version"]
        
        # find the filename
        matching_fnames = [fname for fname in fileset if fname.find("version%d" % v) >= 0]
        fname = None
        if len(matching_fnames) != 1:
            print("ERROR: unexpected number of files for participant %s, test version $s" % (id, v))
            print("       expected 1, found %d" % len(matching_fnames))
            print("      ", matching_fnames)
        else:
            fname = matching_fnames[0]
        
        # map experiment condition to its (0-indexed) chronological session number, version 
        # of the test administered, and name of the .csv file containing the data
        dict[info["condition"]] = {"session_num" : i,
                                "test_version" : v,
                                "file_name" : fname}
    return dict

In [7]:
# store all the current relevant participant info into one dictionary
metadata = {id : categorize(id, infolist, data_files[id]) for id, infolist in sessions.items()}

In [8]:
# print the result to check
if verbose:
    for id, data in metadata.items():
        print("participant", id, ":")
        for condition, sessiondata in data.items():
            print("   ", condition, ":")
            for k, v in sessiondata.items():
                print("      ", k, ":", v)
        print()

The cell below categorizes our stimuli by index into their respective content domains (topics). This can be cross-referenced with our stimuli, which are provided in the replication package.

In [9]:
# given a stimulus number, return the category of the question

# note that we're separating the two positive control categories "shepard-metzler"
# and "PSVT:R II" - see the per-category average scores in the next section for why
def categorize_stimulus(n, print_errors=True):
    if n in [0, 8, 12, 21, 26, 28, 30, 36, 44, 50, 58, 61, 62, 78, 79]:
        if verbose and print_errors:
            print("ERROR: {} not a valid stimulus number".format(n))
        return "ERROR" # ERROR! we have no such stimuli
    
    elif n < 31:
        return "list/array" # inserting to or bubble sorting linked lists or arrays
    elif n < 61:
        return "tree" # rotating or traversing a binary tree
    elif n < 91:
        return "shepard-metzler" # which block is the same as the prompt, but rotated?
    elif n < 121:
        return "PSVT:R II" # a is to a' as b is to which? (rotating blocks again)
    elif n < 151:
        return "code" # what is the output of this code snippet?
    
    elif n < 201:
        if verbose and print_errors:
            print("ERROR: {} not a valid stimulus number".format(n))
        return "ERROR" # ERROR! we have no such stimuli
    
    elif n < 208:
        return "list/array" # again
    elif n < 215:
        return "tree" # again
    
    elif n < 301:
        if verbose and print_errors:
            print("ERROR: {} not a valid stimulus number".format(n))
        return "ERROR" # ERROR! we have no such stimuli
    
    elif n < 315:
        return "tree" # again
    elif n < 317:
        return "code" # again
    elif n < 324:
        return "tree" # again
    elif n < 327:
        return "code" # again
    elif n < 331:
        return "list/array" # again
    elif n < 334:
        return "code"
    
    else:
        if verbose and print_errors:
            print("ERROR: {} not a valid stimulus number".format(n))
        return "ERROR" # ERROR! we have no such stimuli

In [10]:
# given the .csv filename for a session, read the necessary information into a dataframe
def read_session_data(fname):
    # columns to drop
    drop_cols = ["n", "stim_resp.corr_mean", "stim_resp.corr_raw", "stim_resp.corr_std",
                 "stim_resp.rt_raw", "stim_resp.rt_std"]

    # what type to read the columns as
    dtypes = {"figure1" : "str", 
              "corrAns" : "str", 
              "stim_resp.keys_raw" : "str", 
              "stim_resp.rt_mean" : "float64", 
              "order" : "float64"}

    # what to name the columns as
    rename = {"figure1" : "stimulus", # the number of this stimulus in the question bank
              "corrAns" : "correct_response", # the correct answer ("a" or "b")
              "stim_resp.keys_raw" : "actual_response", 
              "stim_resp.rt_mean" : "response_time", # in seconds (I think)
              "order" : "question_num"}

    # read in the data
    df = pd.read_csv(fname, dtype=dtypes).drop(columns=drop_cols) #.dropna()
    
    # find the extra comments at the end and get rid of them
    idx = df.index[df["figure1"] == "extraInfo"].tolist()[0]
    df = df.drop(index=range(idx, df.shape[0])).rename(columns=rename)
    
    # get the stimulus number and question type out of the file name
    df["stimulus"] = [int(s[:-4]) for s in df["stimulus"]]
    df["domain"] = [categorize_stimulus(n) for n in df["stimulus"]]
    
    # cast float to integer
    #df["question_num"] = [int(n) for n in df["question_num"]]
    
    # sort rows in chonological order (i.e. the order they were presented to the participant)
    df = df.sort_values(by="question_num", axis=0).reset_index().drop(columns=["index", "question_num"])

    # condense the response value information
    df["actual_response"] = [s[1] for s in df["actual_response"]]
    df["correct"] = df["actual_response"] == df["correct_response"]
    #df = df.drop(columns=["correct_response", "actual_response"])
    
    # return it
    return df

In [11]:
# do an example for a single file
testdf = read_session_data("test-data-{}/00782/00782_codetms_version3_2023-05-31_15h49.35.491block_1.csv".format(input_version))

# print it out to check
if verbose:
    print("shape:", testdf.shape)
    display(testdf.head())

In [12]:
# now stick everything in a dataframe
main_ds_dict = {}
for id, data in metadata.items():
    participant_ds_dict = {}
    for condition, sessiondata in data.items():
        df = read_session_data(sessiondata["file_name"])
        
        df["session_num"] = sessiondata["session_num"]
        df["test_version"] = sessiondata["test_version"]
        
        participant_ds_dict[condition] = df.stack()
    main_ds_dict[id] = pd.concat(participant_ds_dict, axis=0)

main_ds = pd.concat(main_ds_dict, axis=0)
rename = {"level_0" : "id", "level_1" : "condition", "level_2" : "question"}
main_df = main_ds.unstack().reset_index().rename(columns=rename)

if verbose:
    print("Main dataframe:")
    display(main_df)

In [13]:
# double check that the session number isn't always zero (as it looks from the output above)
if verbose:
    display(main_df.head(104))

## Add the post-test survey results to the dataframe

In [14]:
# make dataframe of the survey data we want to keep

# read in the data, dropping any unnecessary information
drop_cols = ["date", # PID
             "mr_difficulty_change", "programming_difficulty_change", # to difficult to cross-reference
             "discomfort", "other_comments", # not interesting for quantitative data analysis
             "transcriber_notes", "Unnamed: 12", "Unnamed: 13"] # miscellaneous researcher notes

dtypes = {"participant_num" : "str"}

surveydf = pd.read_csv(datadirname + "/post_survey_results.csv", dtype=dtypes).drop(columns=drop_cols)

rename = {"participant_num" : "id", 
          "treatment" : "condition", 
          "session_no" : "session_num", 
          "questions_version" : "test_version"}

surveydf.rename(columns=rename, inplace=True)

surveydf["session_num"] = surveydf["session_num"] - 1 # I zero-indexed this

if verbose:
    display(surveydf.head())

In [15]:
df = main_df.merge(surveydf, on=["id", "condition", "session_num", "test_version"], how="outer", indicator=True)

if verbose:
    display(df.head())

In [16]:
# we expect these to have the same number of rows, but `df` has 3 extra columns with information from the merge
if verbose:
    print(main_df.shape)
    print(df.shape)

In [17]:
if verbose:
    display(df["_merge"].value_counts())    # left_only are observations we don't have survey data for
                                            # right_only should habe 0 entries

    # run this if you want to see them
    display(df.loc[df["_merge"] == "left_only"]["id"].value_counts())

In [18]:
df["_merge"] = df["_merge"] == "both"
df.rename(columns = {"_merge" : "has_survey"}, inplace=True)

In [19]:
main_df = df

## Distinguish mental rotation questions from programming questions

In [20]:
main_df["is_mr"] = main_df["domain"].isin(["shepard-metzler", "PSVT:R II"])

In [21]:
main_df["is_programming"] = main_df["is_mr"] == False

In [22]:
if verbose:
    display(main_df.head())

## Four of the stimuli had incorrectly-coded "correct responses" at the time of data collection, so we flip the values here

In [23]:
actual_correct_answers = {'a' : [], 'b' : [106, 118, 140, 332]}

In [24]:
if verbose:
    for ans, ls in actual_correct_answers.items():
        print("------- correct answer is {} -------".format(ans))
        for stimulus in ls:
            correct_response = main_df[main_df["stimulus"] == stimulus]["correct_response"].iloc[0]
            print("Stimulus #{}: correct answer coded as {}".format(stimulus, correct_response))

In [25]:
if verbose:
    display(main_df[main_df["stimulus"] == 106].head(2))

In [26]:
for ans, ls in actual_correct_answers.items():
    for stimulus in ls:
        ii = main_df["stimulus"] == stimulus
        main_df.loc[ii, "correct_response"] = ans

main_df["correct"] = main_df["correct_response"] == main_df["actual_response"] # have to regenerate this derived column

In [27]:
if verbose:
    for c, ls in actual_correct_answers.items():
        print("------- correct answer is {} -------".format(c))
        for stimulus in ls:
            correct_response = main_df[main_df["stimulus"] == stimulus]["correct_response"].iloc[0]
            print("Stimulus #{}: correct answer coded as {}".format(stimulus, correct_response))

In [28]:
if verbose:
    display(main_df[main_df["stimulus"] == 106].head(2))

In [29]:
if verbose:
    display(main_df.head())

## Anonymize the data

In [30]:
# generate masks
def mask(df, arr, start):
    nums = random.sample(range(len(arr)), len(arr))
    mask = {x : chr(ord(start) + nums[i]) for i, x in enumerate(arr)}
    df.replace(to_replace=mask, inplace=True)
    return mask

In [31]:
if verbose:
    display(mask(pd.DataFrame([]), [1, 2, 3], 'Q')) # example random masking scheme

In [32]:
mask_keys = {}
for k, v in mask_flags.items():
    if v:
        mask_keys[k] = mask(main_df, mask_vals[k], mask_start[k])

In [33]:
if verbose:
    display(mask_keys)

## Fill in gaps in the stimulus indices

Here, we re-index the stimuli in a new `adjusted_stimulus` column. This has no impact on the data analysis, as stimuli are treated categorically. Original stimulus indices are retained in the dataset as `original_stimulus`. The change is purely for aesthetics when data are visualized, so that stimuli of the same domain can be more easily compared. There is a minor additional benefit, in that original stimuli can "give away" the anonymization scheme to someone who knows the data well.

The re-indexing results in stimuli numbered 1-183 in alphabetical order of the domain names. Within each domain, order of stimuli is preserved.

In [34]:
# this cell establishes our knowledge of the indices which don't have a corresponding stimulus
# only the first block should have any indices printed
if verbose:
    # show the missing stimulus indices (there is a large section missing in the middle)
    old_stimuli = pd.unique(main_df["stimulus"])

    print("indices missing from first block, range [1, 150]:\n\t", end='')
    for k in range(1, 151):
        if not k in old_stimuli:
            print("{}".format(k), end=",  ")

    print("\nindices missing from second block, range [201, 214]:\n\t", end='')
    for k in range(201, 215):
        if not k in old_stimuli:
            print("{}".format(k), end=",  ")

    print("\nindices missing from third block, range [301, 333]:\n\t", end='')
    for k in range(301, 333):
        if not k in old_stimuli:
            print("{}".format(k), end=",  ")

    print("\nindices present in between the first and second blocks, range [151, 200]:\n\t", end='')
    for k in range(151, 201):
        if k in old_stimuli:
            print("{}".format(k), end=",  ")

    print("\nindices present in between the second and third blocks, range [215, 300]:\n\t", end='')
    for k in range(215, 300):
        if k in old_stimuli:
            print("{}".format(k), end=",  ")

    #[k for k in range(1, 151) if k not in old_stimuli]

In [35]:
# shift the ranges around
# Note: I'm giving up on very general solutions here (which I tried to stick to above)
# since part of the problem is the gaps in our specific set of stimuli.
if reset_indices:
    domains = pd.unique(main_df["domain"])
    
    if not mask_flags["stimulus_domain"]:
        domains = mask_vals["stimulus_domain"] # so the order doesn't get rearranged
    
    if verbose:
        print(domains)
    
    new_idxs = {}
    start = 1 # 1-indexing, unfortunately
    
    def idx_in_stimulus_domain(idx, dom):
        cat = categorize_stimulus(idx, print_errors=False)
        if cat == "ERROR":
            return False
        if not mask_flags["stimulus_domain"]:
            return cat == dom
        return mask_keys["stimulus_domain"][cat] == dom
    
    # the formatting here will look a little messed up if domains aren't anonymized
    if verbose:
        print("domain \t old starting point \t new starting point \t count")
    
    for dom in domains:
        old_idxs = [k for k in range(1, max(main_df["stimulus"])+1) if idx_in_stimulus_domain(k, dom)]
        new_idxs |= {old_idx : (start + new_idx) for new_idx, old_idx in enumerate(old_idxs)}
        
        if verbose:
            print("{} \t {} \t\t\t {} \t\t\t {}".format(dom, old_idxs[0], new_idxs[old_idxs[0]], len(old_idxs)))
        
        start += len(old_idxs)
    
    if verbose and not start-1 == len(pd.unique(main_df["stimulus"])):
        print("ERROR there are {} unique stimuli, but {} have been stored".format())
    
    main_df.rename(columns={"stimulus" : "original_stimulus"}, inplace=True)
    
    main_df["adjusted_stimulus"] = main_df.apply(lambda row : new_idxs[row["original_stimulus"]], axis=1)

In [36]:
if verbose:
    display(main_df)

## Output the information

Here, we write the fully-compiled dataframe to a `.csv` file.

In [37]:
if verbose:
    display(main_df.head())

In [38]:
# make the output directories
if output:
    path = os.path.realpath("__file__")[:-len("__file__")]

    output_path = os.path.join(path, "processed-data-{}".format(output_version))
    keys_path = os.path.join(output_path, "secret_keys")

    output_fname = "tms-functional-data-v{}.csv".format(output_version)

    try:
        os.mkdir(output_path)
        os.mkdir(keys_path)
    except OSError as error:
        if verbose:
            print(error)

In [39]:
if verbose and output:
    print(path)
    print(output_path)
    print(keys_path)

In [40]:
if output:
    main_df.to_csv(os.path.join(output_path, output_fname), index=False)

In [41]:
if output:
    for k, v in mask_keys.items():
        pd.DataFrame(v.items()).to_csv(os.path.join(keys_path, "{}.csv".format(k)), header=False, index=False)

## Read the data back in to see what it looks like

If you have not run this script with the current output version and with `output=True` at least once, then this cell will throw an exception.

In [42]:
input_df = pd.read_csv(os.path.join(output_path, output_fname), converters={"id" : str})

display(input_df.head())

NameError: name 'output_path' is not defined